In [ ]:
!pip install requests bs4 pandas numpy datetime urllib3 tqdm reachability rpy2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 5.5 MB/s eta 0:00:00


In [ ]:
url = "https://www.courtlistener.com/api/rest/v3/"
api_key = "163cea228fb27936988d579ed72fe787848e1866"


In [ ]:
import pickle
import requests
from urllib.parse import urlencode
from tqdm import tqdm
import time
import requests
from bs4 import BeautifulSoup as beautifulsoup
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from urllib.parse import urlencode
from tqdm import tqdm
import sys
import time
from reachability import reachability
import pickle

def get_data_from_sub_opinions(api_key, url, sub_opinions_ids, excluded_ids, max_queries_per_hour=2000):
    url = url + "opinions/"
    headers = {"Authorization": f"Token {api_key}"}
    opinions_data = []

    start_time = time.time()
    query_count = 0

    with tqdm(total=len(sub_opinions_ids), desc="Fetching sub-opinions data", unit="page") as pbar:
        for seq in sub_opinions_ids:
            params = {
                "id__gte": seq[0],
                "id__lte": seq[1],
                "author__gte": 0,
                "order_by": "id",
                "fields": "plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,opinions_cited,author_id,id,cluster_id",
            }
            while True:
                try:
                    # Check if the rate limit has been reached
                    if query_count >= max_queries_per_hour:
                        elapsed_time = time.time() - start_time
                        if elapsed_time < 3600:  # 3600 seconds = 1 hour
                            sleep_time = 3600 - elapsed_time
                            print(f"Rate limit reached. Waiting for {sleep_time:.2f} seconds...")
                            time.sleep(sleep_time)
                            start_time = time.time()
                            query_count = 0

                    newurl = f"{url}?{urlencode(params, safe=',')}"
                    response = requests.get(newurl, headers=headers)
                    response.raise_for_status()
                    data = response.json()
                    opinions_data.extend(data["results"])
                    if not data["next"]:
                        break
                    params["id__gte"] = data["results"][-1]["id"] + 1
                    pbar.update(1)
                    query_count += 1
                    time.sleep(5)
                except requests.exceptions.HTTPError as err:
                    # If the request fails, wait 3 minutes and try again with the same parameters
                    print(err)
                    time.sleep(180)
                    continue
                except requests.exceptions.ConnectionError as err:
                    reachable = reachability(timeout=None)
                    while not reachable.is_online():
                        print("Waiting for internet connection...")
                        time.sleep(120)
                    continue
                except Exception as err:
                    print("Other error: ", err)
                    break
    return opinions_data

# Open the pickled file
with open('clusters_data.pickle', 'rb') as f:
    clusters = pickle.load(f)

from google.colab import drive
drive.mount('/content/drive')

# Extract the sub_opinions values from each dictionary
sub_opinions = [item['sub_opinions'] for item in clusters if 'sub_opinions' in item]

# Flatten the list of lists and convert to a set to get unique values
unique_values = set(x for sublist in sub_opinions for x in sublist)

# Convert the set to a sorted list
result = sorted(list(unique_values))

# Find the sequences of numbers
sequences = []
current_sequence = [result[0]]

for i in range(1, len(result)):
    if result[i] == result[i-1] + 1:
        current_sequence.append(result[i])
    else:
        sequences.append(current_sequence)
        current_sequence = [result[i]]

sequences.append(current_sequence)

# Format the sequences
formatted_sequences = []
for seq in sequences:
    if len(seq) == 1:
        formatted_sequences.append([seq[0], seq[0]])
    else:
        formatted_sequences.append([seq[0], seq[-1]])

# Sort the formatted sequences by the difference between the two numbers in descending order
sorted_sequences = sorted(formatted_sequences, key=lambda x: x[1] - x[0], reverse=True)

# Call the get_data_from_sub_opinions function with the sorted sequences
excluded_ids = []
opinions_data = get_data_from_sub_opinions(api_key, url, sorted_sequences, excluded_ids, max_queries_per_hour=2000)

# Save the fetched opinions data as a pickle file
with open('/content/drive/MyDrive/opinions_data.pickle', 'wb') as f:
    pickle.dump(opinions_data, f)

print("Fetched opinions data saved as 'opinions_data.pickle'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fetching sub-opinions data:   0%|          | 1/18050 [00:02<11:16:36,  2.25s/page]

502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/opinions/?id__gte=4454847&id__lte=4454851&author__gte=0&order_by=id&fields=plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,opinions_cited,author_id,id,cluster_id
502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/opinions/?id__gte=4398523&id__lte=4398526&author__gte=0&order_by=id&fields=plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,opinions_cited,author_id,id,cluster_id
502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/opinions/?id__gte=4435007&id__lte=4435010&author__gte=0&order_by=id&fields=plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,opinions_cited,author_id,id,cluster_id
502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/opinions/?id__gte=9297448&id__lte=9297451&author__gte=0&order_by=id